In [213]:
import gzip, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict

def is_valid(bc):
    return (bc[4:6]=='TG'
            and bc[10:12]=='CA' 
            and bc[16:18]=='AC' 
            and bc[22:24]=='GA' 
            and bc[28:30]=='GT'
            and bc[34:36]=='AG')

def in_filtered_list(cell_bc, filtered_cell_barcodes):
    num_Ns = sum([c=='N' for c in cell_bc])
    if num_Ns > 1: return False
    elif num_Ns == 1: return np.any([cell_bc.replace('N',c) in filtered_cell_barcodes for c in 'ACTG'])
    else: return cell_bc in filtered_cell_barcodes

In [ ]:
# set cell type information and larry fasta to get larry clone
#celltype file path, get from LARRY_D4, etc.
f_p = "/autofs/bal31/jhsu/home/projects/sc/data/hemo_paper_larry/larry-maester-d4/sc/2_bam/outs/raw_celltype.csv"
# larry file path
fastq_path = '/autofs/bal36/jhsu/sc/larry/SugimuraRR_UCSO_CPOS-240606-RRS-23942a/SugimuraRR_UCSO_CPOS-240606-RRS-23942a/primary_seq/larry-amplicon_{}.fastq.gz'
sample = 'Larry_1'

In [ ]:
_celltype_p = f_p
_tmp_df = pd.read_csv(_celltype_p, header=0)

filtered_cell_barcodes = list(_tmp_df['Unnamed: 0'].values)
filtered_cell_barcodes_set = set(filtered_cell_barcodes)

In [215]:
larry_prefix = 'GTTGCTAGGAGAGACCATATG'
N_READS = 10
N_UMIS = 3
N_HAMMING = 3

output_prefix = 'Larry_1'

In [ ]:
counts = {}

# fastq_path = '/autofs/bal36/jhsu/sc/larry/SugimuraRR_UCSO_CPOS-240606-RRS-23942a/SugimuraRR_UCSO_CPOS-240606-RRS-23942a/primary_seq/larry-amplicon_{}.fastq.gz'
# sample = 'Larry_1'

R1 = gzip.open(fastq_path.format('1'))
R2 = gzip.open(fastq_path.format('2'))
counter = 0
start_time = time.time()
while True:
    counter += 1
    if counter % 1000000 == 0: print(fastq_path+ ': Processed {} lines in {} seconds'.format(counter, time.time()-start_time))
    try:
        r1_line = R1.readline().decode('utf-8')
        r2_line = R2.readline().decode('utf-8')
    except:
        print('ERROR extracting {}'.format(fastq_path))
        break
    if r2_line == '': break
    if r2_line[0] in '@+': continue
    if larry_prefix in r2_line:
        larry_bc = r2_line.split(larry_prefix)[1][:40]
        cell_bc = r1_line[:16]+'-1'
        umi = r1_line[16:24]
        if is_valid(larry_bc) and in_filtered_list(cell_bc, filtered_cell_barcodes):
            combo = (sample, cell_bc, umi, larry_bc)
            if combo in counts:
                counts[combo] += 1
            else:
                counts[combo] = 1

In [ ]:
num_reads = [v for k,v in counts.items()]
plt.hist(np.log(num_reads)/np.log(10), bins=50)
plt.axvline(np.log(N_READS)/np.log(10),c='k')
plt.xticks(range(5),np.logspace(0,4,5))
plt.text(np.log(N_READS)/np.log(10)*1.1,10**6,'N_READS cutoff', fontsize=12)
plt.yscale('log')

counts_filtered = {k:v for k,v in counts.items() if v >= N_READS}
print('Retaining '+repr(len(counts_filtered))+ ' out of '+repr(len(counts))+' (Sample,Cell-BC,UMI,GFP-BC) combinations')


In [ ]:
def hamming(bc1,bc2): return np.sum([x1 != x2 for x1,x2 in zip(bc1,bc2)])

all_gfp_bcs = sorted(set([k[3] for k in counts_filtered]))
good_gfp_bcs = []
bc_map = {}
for i,bc1 in enumerate(all_gfp_bcs):
    if i > 0 and i % 500 == 0: print('Mapped '+repr(i)+' out of '+repr(len(all_gfp_bcs))+' barcodes')
    mapped = False
    for bc2 in good_gfp_bcs:
        if hamming(bc1,bc2) <= N_HAMMING:
            mapped = True
            bc_map[bc1] = bc2
            break
    if not mapped:
        good_gfp_bcs.append(bc1)

print('\nCollapsed '+repr(len(bc_map))+' barcodes')
for bc in good_gfp_bcs: bc_map[bc] = bc


In [ ]:
# cell_data = {}
# for sample,paths in sample_paths.items():
#     filtered_cell_barcodes = gzip.open(paths['filtered_cell_barcodes_path']).read().decode('utf-8').split('\n')
#     for cell_bc in filtered_cell_barcodes:
#         cell_data[(sample,cell_bc)] = {}

cell_data = {}
for cell_bc in filtered_cell_barcodes:
    cell_data[(sample,cell_bc)] = {}


for sample,cell_bc,umi,larry_bc in counts_filtered.keys():
    if (sample,cell_bc) in cell_data:
        if not larry_bc in cell_data[(sample,cell_bc)]:
            cell_data[(sample,cell_bc)][larry_bc] = 0
        cell_data[(sample,cell_bc)][larry_bc] += 1

num_cells_with_barcode = np.zeros(20)
for larry_bc_counts in cell_data.values():
    if len(larry_bc_counts)>0:
        num_cells_with_barcode[:np.min([20,np.max(list(larry_bc_counts.values()))])] += 1
efficiency = num_cells_with_barcode / len(cell_data)
plt.plot(range(20),efficiency)
plt.plot([N_UMIS,N_UMIS],[np.min(efficiency),np.max(efficiency)],'-k',linewidth=2)
plt.text(N_UMIS*1.1,np.max(efficiency)*.95,'UMI cutoff',fontsize=14)

final_BCs = {}
for k,larry_bc_counts in cell_data.items():
    final_BCs[k] = '-'.join(sorted([k for k,v in larry_bc_counts.items() if v >= N_UMIS]))
print('\nFinal annotation has '+repr(len(set(final_BCs.values())))+' clones in '+repr(len([k for k,v in final_BCs.items() if len(v)>0]))+' cells')


In [ ]:
output = []
for cell_bc in filtered_cell_barcodes:
    output.append(sample+','+cell_bc+','+final_BCs[(sample,cell_bc)])
open(output_prefix+'.larry_clones.csv','w').write('\n'.join(output))

In [ ]:
BC_set = sorted(set([final_BCs[(k, v)] for k, v in final_BCs if final_BCs[(k, v)] != '']))
clone_mat = np.zeros((len(final_BCs),len(BC_set)))
for i,bc in enumerate(final_BCs.values()):
#     print(i, bc)
    if bc != '':
        j = BC_set.index(bc)
        clone_mat[i,j] = 1
clone_mat = np.array(clone_mat,dtype=int)
np.savetxt('clone_mat.csv',clone_mat,delimiter=',',fmt='%i')
# open('barcode_list.txt','w').write('\n'.join(final_BCs));


In [ ]:
n_l = []
i = 0
j = 0
for k, v in final_BCs:
#     print(k, v, final_BCs[(k, v)])
    j += 1
    if final_BCs[(k, v)] != '':
        n_l.append(v)
        i += 1
print(i, j)
len(set(n_l))

In [ ]:
ax = pd.DataFrame([i.split(",") for i in output]).groupby(2).size().hist(bins=200)
ax.set_xlim((3,16))
ax.set_ylim((0, 200))

In [ ]:
ax = pd.DataFrame([i.split(",") for i in output]).groupby(2).size().hist(bins=200)
# ax.set_xlim((2,10))